In [24]:
import polars as pl

In [25]:
games_df = pl.read_csv("games_df.csv")
games_df = games_df.filter(
    (pl.col("forbidMoving") == True)
    & (pl.col("forbidZooming") == False)
    & (pl.col("forbidRotating") == False)
    & (pl.col("roundCount") == 5)
    & (pl.col("roundTime") == 40)
    & (pl.col("mapSlug") == "world")
)
games_df.head(2)

game_id,forbidMoving,forbidZooming,forbidRotating,roundCount,mapSlug,roundTime,isSinglePlayer,scoringType,context,date
str,bool,bool,bool,i64,str,i64,bool,str,str,str
"""43b82fcf-29d8-4daf-84e9-80e029…",true,false,false,5,"""world""",40,false,"""Unknown""","""""","""2024-10-30T18:06:54.013Z"""
"""14cf63a7-1289-4b09-9ab8-75eb4f…",true,false,false,5,"""world""",40,false,"""Unknown""","""""","""2024-10-30T18:11:55.588Z"""


In [26]:
answers_df = pl.read_csv("answers_df.csv")
answers_df = answers_df.filter(
    pl.col("gameId").is_in(games_df["game_id"].unique())
)
answers_df.head(2)

gameId,roundNumber,lat,lng,country_code
str,i64,f64,f64,str
"""43b82fcf-29d8-4daf-84e9-80e029…",1,37.604549,126.933647,"""KR"""
"""43b82fcf-29d8-4daf-84e9-80e029…",2,55.926456,-4.126496,"""GB"""


In [27]:
guesses_df = pl.read_csv("guesses_df.csv")
guesses_df = guesses_df.filter(
    pl.col("gameId").is_in(games_df["game_id"].unique())
)
guesses_df.head(2)

roundNumber,lat,lng,distance,time,score,wasCorrect,gameId,playerName,code,playerId,payload,timestamp,lobby,countryGuess,coordinateGuess,battleRoyaleGameState,battleRoyalePlayer,duel,bullseye,player,country_code
i64,f64,f64,f64,i64,i64,bool,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
3,51.47018,5.786652,117653.422716,40,4621,false,"""ea8d34ee-d2c9-4283-834e-f0438f…","""moxxiq""","""LiveChallengeLeaderboardUpdate""",null,null,"""2024-08-21T18:47:34.5599291Z""",null,null,null,null,null,null,null,"""moxxiq""","""NL"""
3,-34.665525,-58.416663,3.9950e6,34,343,false,"""2d460ac2-f967-420b-9867-cefe08…","""Ishenko01y""","""LiveChallengeLeaderboardUpdate""",null,null,"""2024-12-11T18:39:45.8073149Z""",null,null,null,null,null,null,null,"""Ishenko01y""","""AR"""


In [34]:
cols = ["gameId", "roundNumber", "country_code"]
df = guesses_df[cols + ["score"]].join(answers_df[cols], on=["gameId", "roundNumber"], suffix="_answer")
df.head(2)

gameId,roundNumber,country_code,score,country_code_answer
str,i64,str,i64,str
"""ea8d34ee-d2c9-4283-834e-f0438f…",3,"""NL""",4621,"""NL"""
"""2d460ac2-f967-420b-9867-cefe08…",3,"""AR""",343,"""PE"""


In [35]:
df.filter(pl.col("country_code") != pl.col("country_code_answer")).with_columns(
    guess_answer=pl.when(pl.col("country_code") <= pl.col("country_code_answer"))
    .then(pl.col("country_code") + "_" + pl.col("country_code_answer"))
    .otherwise(pl.col("country_code_answer") + "_" + pl.col("country_code"))
)["guess_answer"].value_counts(sort=True)[:10]

guess_answer,count
str,u32
"""CA_US""",34
"""AT_DE""",31
"""GB_IE""",23
"""SZ_ZA""",20
"""ID_MY""",19
"""AU_US""",16
"""EE_FI""",16
"""EE_LT""",16
"""AU_NZ""",16


In [36]:
def get_correct(a):
    print(a[0], a[1])
    return a.filter(pl.col("country_code") == a[1][0]).shape[0]

df = df.with_columns(
    guessed=pl.col("country_code") == pl.col("country_code_answer")
)
df_agg = df.group_by("country_code_answer").agg(
    total=pl.col("country_code_answer").count(),
    guessed=pl.col("guessed").filter(pl.col("guessed") == True).count(),
    not_guessed=pl.col("guessed").filter(pl.col("guessed") == False).count(),
    avg_score=pl.col("score").mean()
).with_columns(
    ratio=pl.col("guessed") / pl.col("total")
)
df_agg.filter(
    (pl.col("total") > 40)
    & (pl.col("ratio") < 0.2)
).sort(by="total", descending=True)[:5]

country_code_answer,total,guessed,not_guessed,avg_score,ratio
str,u32,u32,u32,f64,f64
"""AR""",78,13,65,1133.153846,0.166667
"""FI""",65,12,53,2742.615385,0.184615
"""ZA""",64,10,54,1345.8125,0.15625
"""BE""",61,10,51,3401.803279,0.163934
"""CL""",59,7,52,1100.813559,0.118644


In [37]:
df_agg.filter(pl.col("total") > 10).sort(by="ratio", descending=True)[:5]

country_code_answer,total,guessed,not_guessed,avg_score,ratio
str,u32,u32,u32,f64,f64
"""KR""",30,26,4,4054.033333,0.866667
"""UA""",35,28,7,3978.314286,0.8
"""GR""",30,23,7,3949.1,0.766667
"""NL""",62,43,19,4186.548387,0.693548
"""IN""",28,18,10,2396.892857,0.642857


In [44]:
df_agg.filter((pl.col("total") > 10) & (pl.col("ratio") > 0.4)).sort(by="avg_score", descending=False)[:5]

country_code_answer,total,guessed,not_guessed,avg_score,ratio
str,u32,u32,u32,f64,f64
"""AU""",110,45,65,1264.8,0.409091
"""CA""",71,30,41,1727.267606,0.422535
"""NZ""",43,20,23,1744.860465,0.465116
"""US""",88,55,33,1866.306818,0.625
"""IN""",28,18,10,2396.892857,0.642857


In [32]:
df.with_columns(
    guessed=pl.col("country_code") == pl.col("country_code_answer")
).group_by("country_code").agg(
    total=pl.col("country_code").count(),
    guessed=pl.col("guessed").filter(pl.col("guessed") == True).count(),
    not_guessed=pl.col("guessed").filter(pl.col("guessed") == False).count()
).with_columns(
    ratio=pl.col("guessed") / pl.col("total")
).sort(by="total", descending=True)[:5]

country_code,total,guessed,not_guessed,ratio
str,u32,u32,u32,f64
"""US""",140,55,85,0.392857
"""MX""",106,27,79,0.254717
"""FR""",102,24,78,0.235294
"""AT""",86,53,33,0.616279
"""AU""",82,45,37,0.54878
